In [1]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipp

True

In [2]:
import pandas as pd

In [3]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [4]:
import os
workspace_path = '/gdrive/My Drive/DA_텀프로젝트' 
data_path = os.path.join(workspace_path, 'data') 

In [5]:
train = pd.read_csv(os.path.join(workspace_path, 'DA_텀프로젝트_데이터/2. 소설작가분류/train.csv'))
train.head()

,index,text,author
0,0,"He was almost choking. There was so much, so m...",3
1,1,"“Your sister asked for it, I suppose?”",2
2,2,"She was engaged one day as she walked, in per...",1
3,3,"The captain was in the porch, keeping himself ...",4
4,4,"“Have mercy, gentlemen!” odin flung up his han...",3


In [6]:
test = pd.read_csv(os.path.join(workspace_path, 'DA_텀프로젝트_데이터/2. 소설작가분류/test_x.csv'))
test.head()

,index,text
0,0,“Not at all. I think she is one of the most ch...
1,1,"""No,"" replied he, with sudden consciousness, ""..."
2,2,As the lady had stated her intention of scream...
3,3,“And then suddenly in the silence I heard a so...
4,4,His conviction remained unchanged. So far as I...


In [7]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

In [8]:
stopWords = set(stopwords.words("english"))
Tfidf_Vectorizer = TfidfVectorizer(tokenizer=word_tokenize, stop_words=stopWords, ngram_range=(1, 3),min_df=10)
X_train = Tfidf_Vectorizer.fit_transform(train['text'])

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [9]:
X_test = Tfidf_Vectorizer.transform(test['text'])
Y_train = train["author"]

In [10]:
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

In [11]:
skf = StratifiedKFold(n_splits=5)

Y_pred_Accuracy = np.zeros((54879, 5))
Y_pred_Result = np.zeros((19617, 5))
for i, (train_index, test_index) in enumerate(skf.split(X_train, Y_train)):
    logi = LogisticRegression()
    logi.fit(X_train[train_index], Y_train[train_index])
    Y_pred_Accuracy[test_index, :] = logi.predict_proba(X_train[test_index])
    Y_pred_Result = logi.predict_proba(X_test) 

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [12]:
print(f'Accuracy : {accuracy_score(Y_train, np.argmax(Y_pred_Accuracy, axis=1))}')

Accuracy : 0.7930902531022795


In [13]:
submission = pd.read_csv(os.path.join(workspace_path+'/DA_텀프로젝트_데이터/2. 소설작가분류/', 'sample_submission.csv'), index_col=0)
submission[submission.columns] = Y_pred_Result
submission.to_csv(os.path.join(workspace_path, '텀프로젝트_2_결과.csv'))